# TP 1: LDA/QDA y optimización matemática de modelos

# Librerias

In [4]:
import numpy as np
import numpy.linalg as LA

import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal

from base.qda import QDA, TensorizedQDA, FastQDA, EfficientQDA
from base.cholesky import QDA_Chol1, QDA_Chol2, QDA_Chol3
from utils.bench import Benchmark
from utils.datasets import (get_iris_dataset, get_letters_dataset, 
                            get_penguins_dataset, get_wine_dataset,
                            label_encode)

# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda *fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?

 Paraleliza sobre las $k$ clases.

 El método _fit_params(X,y) de la clase derivada `TensorizedQDA` obtiene de la clase base `QDA` las inversas de las matrices de covarianzas y vectores de medias de cada clase como dos listas, cada elemento correspondiente a una clase.
 - Tipo `means` e `inv_covs`: <class 'list'>.
 - Cantidad de elementos de las listas: $k$ (uno por clase).
 - Formato elementos lista `means`: (p, 1).
 - Formato elementos lista `inv_covs`: (p, p).

 Luego, utiliza el método `stack` de numpy para "apilar" los elementos de las listas en un tensor de una dimensión adicional a la de los elementos. El primer eje del tensor corresponde al índice de la clase aplilada (`batch`).
 - Tipo `tensor_means` y `tensor_inv_covs`: <class 'numpy.ndarray'>.
 - Formato `tensor_means`: (k, p, 1).
 - Formato `tensor_inv_covs`: (k, p, p).

 Este arreglo hace que la multiplicación matricial (@) interprete las dos últimas dimensiones del tensor como matrices a multiplicar y las restantes como índices de lote (batch), realizando las operaciones en paralelo sobre las clases. De modo análogo, np.linalg.det calcula los determinantes en paralelo reconociendo los lotes de matrices.


2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

Los shapes de `tensor_inv_covs` y `tensor_means` y como son obtenidos se analizaron en el punto anterior.

`TensorizedQDA` optimiza paralelizando el cálculo de la log-condicional para cada clase dada una observación. Para eso sobreescribe el método `_predict_one` de la clase base, eliminando el lazo `for` donde se realizaba el cálculo de esta probabilidad para cada clase y definiendo el método `_predict_log_conditionals` donde este cálculo se realiza en paralelo a partir de los tensores `tensor_inv_covs` y `tensor_means` mencionados.

Paso a paso:

1) Entrenamiento

Se obtienen las listas de medias e inversas de matrices de covarianzas de cada clase de idéntico modo que en `QDA`. Se sobreescribe el método `_fit_params` para llamar primero al de la clase base QDA y luego generar las versiones tensorizadas.

```python
def _fit_params(self, X, y):
        super()._fit_params(X,y)
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)
```

Basicamente, el entrenamiento en sí es idéntico en ambos casos.

2) Predicción:

Se sobreescribe el método `_predict_one` de la clase `BaseBayesianClassifier`eliminando el lazo `for` y devolviendo el indice de clase como el mayor de la suma del array de probabilidades a priori y lel array de log-condicionales de cada clase obtenidas de forma paralela en el nuevo método `_predict_log_conditionals`.

```python
def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x)) 
```
En el método `_predict_log_conditionals` la resta

```python
def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x
```

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.

4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.

5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$
queda a preferencia del alumno cuál usar.

6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.

7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [6]:
# Levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total

X_full, y_full = get_wine_dataset()

# Encodeamos a número las clases

y_full_encoded = label_encode(y_full)

# Generamos el benchmark

b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 1000,
    warmup = 200,
    mem_runs = 200,
    test_sz = 0.3,
    same_splits = False
)

# Bencheamos implementaciones

to_bench = [QDA, TensorizedQDA, FastQDA, EfficientQDA]

for model in to_bench:
    b.bench(model)

# Hacemos un summary

b.summary()

# Son muchos datos! nos quedamos con un par nomás

summ = b.summary()

# Como es un pandas DataFrame, subseteamos columnas fácil

summ[['train_median_ms', 'test_median_ms', 'mean_accuracy']]

Benching params:
Total runs: 1400
Warmup runs: 200
Peak Memory usage runs: 200
Running time runs: 1000
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


QDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

QDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

TensorizedQDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

TensorizedQDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

FastQDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

FastQDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

EfficientQDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

EfficientQDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

,train_median_ms,test_median_ms,mean_accuracy
model,,,
QDA,1.03050,7.84060,0.983963
TensorizedQDA,0.98980,2.90765,0.984074
FastQDA,1.08825,0.22655,0.984796
EfficientQDA,0.95255,0.20755,0.984315


## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?
7. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.
8. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?
9. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.
13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.
13. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Importante:

Las métricas que se observan al realizar benchmarking son muy dependientes del código que se ejecuta, y por tanto de las versiones de las librerías utilizadas. Una forma de unificar esto es utilizando un gestor de versiones y paquetes como _uv_ o _Poetry_, otra es simplemente usando una misma VM como la que provee Colab.

**Cada equipo debe informar las versiones de Python, NumPy y SciPy con que fueron obtenidos los resultados. En caso de que sean múltiples, agregar todos los casos**. La siguiente celda provee una ayuda para hacerlo desde un notebook, aunque como es una secuencia de comandos también sirve para consola.

In [ ]:
# Version que deberia funcionar en todos los entornos

import sys
print("Python:", sys.version)

# importlib.metadata es estándar desde Python 3.8; en Colab también está.
try:
    from importlib.metadata import version, PackageNotFoundError
except Exception:
    from importlib_metadata import version, PackageNotFoundError  # backport, por si acaso

for pkg in ["numpy", "scipy"]:
    try:
        print(f"{pkg}:", version(pkg))
    except PackageNotFoundError:
        print(f"{pkg}: NO INSTALADO")

Python: 3.12.7 | packaged by conda-forge | (main, Oct  4 2024, 15:47:54) [MSC v.1941 64 bit (AMD64)]
numpy: 1.26.4
scipy: 1.15.2
